In [1]:
import pandas as pd
import os
import re
import numpy as np

In [2]:
ls data 

0_reviews_Automotive_5_word_vecs.npy
0_reviews_Beauty_5_review_embeds.npy
0_reviews_Beauty_5_word_vecs.npy
0_reviews_Digital_Music_5_word_vecs.npy
0_reviews_Musical_Instruments_5_word_vecs.npy
10_reviews_Beauty_5_review_embeds.npy
10_reviews_Beauty_5_word_vecs.npy
10_reviews_Digital_Music_5_word_vecs.npy
10_reviews_Musical_Instruments_5_word_vecs.npy
11_reviews_Beauty_5_review_embeds.npy
11_reviews_Beauty_5_word_vecs.npy
11_reviews_Digital_Music_5_word_vecs.npy
11_reviews_Musical_Instruments_5_word_vecs.npy
12_reviews_Beauty_5_review_embeds.npy
12_reviews_Beauty_5_word_vecs.npy
12_reviews_Musical_Instruments_5_word_vecs.npy
13_reviews_Beauty_5_review_embeds.npy
13_reviews_Beauty_5_word_vecs.npy
13_reviews_Musical_Instruments_5_word_vecs.npy
14_reviews_Beauty_5_review_embeds.npy
14_reviews_Beauty_5_word_vecs.npy
14_reviews_Musical_Instruments_5_word_vecs.npy
15_reviews_Beauty_5_review_embeds.npy
15_reviews_Beauty_5_word_vecs.npy
15_reviews_Musical_Instruments_5_word_vecs.npy
16_reviews_

In [6]:
cleaned_files = [ f for f in os.listdir("data") if f.endswith(".csv") and f.startswith("reviews_") and "word_vecs" not in f]
cleaned_files

['reviews_Amazon_Instant_Video_5.csv',
 'reviews_Apps_for_Android_5.csv',
 'reviews_Automotive_5.csv',
 'reviews_Baby_5.csv',
 'reviews_Beauty_5.csv',
 'reviews_Cell_Phones_and_Accessories_5.csv',
 'reviews_Clothing_Shoes_and_Jewelry_5.csv',
 'reviews_Digital_Music_5.csv',
 'reviews_Electronics_5_0.csv',
 'reviews_Electronics_5_1.csv',
 'reviews_Electronics_5_2.csv',
 'reviews_Electronics_5_3.csv',
 'reviews_Electronics_5_4.csv',
 'reviews_Electronics_5_5.csv',
 'reviews_Electronics_5_6.csv',
 'reviews_Grocery_and_Gourmet_Food_5.csv',
 'reviews_Health_and_Personal_Care_5.csv',
 'reviews_Home_and_Kitchen_5_0.csv',
 'reviews_Home_and_Kitchen_5_1.csv',
 'reviews_Musical_Instruments_5.csv',
 'reviews_Office_Products_5.csv',
 'reviews_Patio_Lawn_and_Garden_5.csv',
 'reviews_Pet_Supplies_5.csv',
 'reviews_Sports_and_Outdoors_5.csv',
 'reviews_Tools_and_Home_Improvement_5.csv']

In [7]:

def one_hot_label(label):
    if label==0:
        return np.array([1,0])
    else:
        return np.array([0,1])

In [5]:


# batchSize = 1024
# lstmUnits = 64
# numClasses = 2
# iterations = 100
# maxSeqLength = 50



def review_to_bow_vector(rev):
    rev = re.sub('[^-a-zA-Z0-9_ -]+', '', rev)
    split_words = rev.split()
    
    firstFile = np.zeros((maxSeqLength), dtype='int32')
    indexCounter = 0
    for word in split_words[0:250]:
        try: 
            firstFile[indexCounter] = wordsList.index(word)
        except ValueError:
            firstFile[indexCounter] = 399999 #Vector for unknown words
        indexCounter = indexCounter + 1
    return firstFile


# load the GLOVE arrays, smaller than w2v google, hopefully no perfomance drop
wordsList = np.load('data/wordsList.npy')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('utf-8') for word in wordsList] 
wordVectors = np.load('data/wordVectors.npy')

#good_files = ["data/"+f for f in os.listdir("data") if ".csv" in f]
good_files = ["data/reviews_Musical_Instruments_5.csv", "data/reviews_Automotive_5.csv", "data/reviews_Amazon_Instant_Video_5.csv"]
file_count = -1


In [23]:
frames = []

for f in cleaned_files:
    print(f)
    frame = pd.read_csv("data/"+f, encoding='utf-8')
    print("frame shape", frame.shape, frame.columns)
    
    frame = frame[(frame.clean_text.notnull()) & (frame.clean_text.str.len() > 100)]
    print("frame shape", frame.shape)
    frames.append(frame)
    
data = pd.concat(frames)
print("final shape", data.shape)
    
    

reviews_Amazon_Instant_Video_5.csv
frame shape (32939, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (31427, 2)
reviews_Apps_for_Android_5.csv
frame shape (667816, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (592482, 2)
reviews_Automotive_5.csv
frame shape (19043, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (18299, 2)
reviews_Baby_5.csv
frame shape (143537, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (139679, 2)
reviews_Beauty_5.csv
frame shape (176254, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (169808, 2)
reviews_Cell_Phones_and_Accessories_5.csv
frame shape (173000, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (162479, 2)
reviews_Clothing_Shoes_and_Jewelry_5.csv
frame shape (248252, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (234449, 2)
reviews_Digital_Music_5.csv
frame shape (57917, 2) Index(['clean_text', 'sentiment'], dtype='obje

In [24]:
import sklearn.utils
print(data.iloc[1])

data = sklearn.utils.shuffle(data, random_state=1)
print(data.iloc[1])

clean_text    i highly recommend this series it is a must fo...
sentiment                                                     1
Name: 1, dtype: object
clean_text    these speaker have a small footprint and big s...
sentiment                                                     1
Name: 125914, dtype: object


In [57]:
start_index = 0
split_size = 16384
shuffled_frames = []
while start_index+split_size < data.shape[0]:
    shuffled_frames.append(data[start_index:start_index+split_size])
    start_index += split_size


In [ ]:
i = 6

## Glove embeddings

In [39]:

for i in range(40, 50):
    file_name = "reviews_shuffled_" + str(i)
    label_file_name = "labels_shuffled_" + str(i)
    batch = shuffled_frames[i]
    print("batch selected",i, batch.shape)
    vecs = batch.clean_text.map(review_to_bow_vector)


    arr = np.zeros((split_size, maxSeqLength))
    labels = np.zeros((split_size, numClasses))

    for j, v in enumerate(vecs):
        arr[j,] = v
        labels[j,] = one_hot_label(batch.sentiment.iloc[j])

    np.save("data/shuffled/" + file_name, arr)
    np.save("data/shuffled/" + label_file_name, labels)


batch selected 40 (16384, 2)


IndexError: index 50 is out of bounds for axis 0 with size 50

## Google news embedding vectors


In [8]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format("w2v/GoogleNews-vectors-negative300.bin", binary=True)

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [10]:
i = 0
for word, vocab_obj in model.vocab.items():
    
    print(word, vocab_obj.count)
    
    if i % 20 == 19:
        break
    
    i += 1

</s> 3000000
in 2999999
for 2999998
that 2999997
is 2999996
on 2999995
## 2999994
The 2999993
with 2999992
said 2999991
was 2999990
the 2999989
at 2999988
not 2999987
as 2999986
it 2999985
be 2999984
from 2999983
by 2999982
are 2999981


In [11]:
NUM_WORDS = 200000

In [12]:
def is_valid_word(w):
    """
        Only want lower case words that aren't stopwords and arent tags or other nonsense like ####
    """
    if not w.lower() == w:
        return False
#     if w in stop_words:
#         return False
    if re.search("[^(\w|\'|\-)]", w):
        return False
#     if not w in model.wv.vocab:
#         # shouldnt have to add this but for some reason it makes a difference, where are these words coming from?
#         return False
    
    return True
     

In [13]:
vocab_counts = [(word, vocab_obj.count) for  (word, vocab_obj) in model.vocab.items() if is_valid_word(word)]
vocab_counts = sorted(vocab_counts, key=lambda x:x[1], reverse=True)
print("found counts")
# needs a list for ordering
final_vocab = [v[0] for v in vocab_counts[0:NUM_WORDS]]
print("vocab set")
# get a lookup for O[1] access
final_vocab_lookup = {v:final_vocab.index(v) for v in final_vocab}


found counts
vocab set


In [14]:
reverse_lookup = {v: k for k, v in final_vocab_lookup.items()}

In [15]:
count = 0
for v in final_vocab_lookup:
    print(v)
    count += 1
    if count % 10 == 9:
        break

in
for
that
is
on
with
said
was
the


In [16]:
final_vocab[0:20]

['in',
 'for',
 'that',
 'is',
 'on',
 'with',
 'said',
 'was',
 'the',
 'at',
 'not',
 'as',
 'it',
 'be',
 'from',
 'by',
 'are',
 'have',
 'he',
 'will']

In [17]:
word_vectors = np.zeros((NUM_WORDS, 300))

for i, v in enumerate(final_vocab):
    if v not in model.wv.vocab:
        print("huh", v)
    word_vectors[i,] = model.wv[v]
    
np.save("data/w2v_vectors.npy", word_vectors)

In [19]:
import collections
from imblearn.over_sampling import SMOTE


In [66]:
review_sizes = collections.defaultdict(int)
first = True
def review_to_w2v_vector(rev):
    split_words = rev.split()
    
    words_in_order = np.random.randint(len(final_vocab), size=maxSeqLength, dtype='int32')
    indexCounter = 0
    good_words = [w for w in split_words if w in final_vocab_lookup]
    review_sizes[len(good_words)] += 1
    
    for word in good_words:
        words_in_order[indexCounter] = final_vocab_lookup[word]
        indexCounter += 1
        if indexCounter >= maxSeqLength:
            break
    return words_in_order


for i in range(75, 100):
    
    file_name = "balanced_w2vreviews_" + str(i)
    label_file_name = "balanced_w2vlabels_" + str(i)
    batch = shuffled_frames[i]
    
    
    print("batch selected",i, batch.shape)
    vecs = batch.clean_text.map(review_to_w2v_vector)


    arr = np.zeros((split_size, maxSeqLength))
    labels = np.zeros((split_size, numClasses))

    for j, v in enumerate(vecs):
        arr[j,] = v
        labels[j,] = one_hot_label(batch.sentiment.iloc[j])
        
        
    if not first:
        # dont oversample, test set
        sm = SMOTE(ratio='minority', random_state=42)
        arr, new_labels = sm.fit_sample(arr, batch.sentiment.values)
        labels = np.zeros((arr.shape[0], numClasses))
        for i, l in enumerate(new_labels):
            labels[i,] = one_hot_label(new_labels[i])
    else:
        first = False
        file_name = file_name.replace("balanced", "test")
        label_file_name = label_file_name.replace("balanced", "test")
    
    print("saving arr", arr.shape)
    np.save("data/vecs/" + file_name, arr)
    np.save("data/vecs/" + label_file_name, labels)

batch selected 75 (16384, 2)
saving arr (16384, 50)
batch selected 76 (16384, 2)
saving arr (28678, 50)
batch selected 77 (16384, 2)
saving arr (28734, 50)
batch selected 78 (16384, 2)
saving arr (28662, 50)
batch selected 79 (16384, 2)
saving arr (28764, 50)
batch selected 80 (16384, 2)
saving arr (28790, 50)
batch selected 81 (16384, 2)
saving arr (28592, 50)
batch selected 82 (16384, 2)
saving arr (28638, 50)
batch selected 83 (16384, 2)
saving arr (28690, 50)
batch selected 84 (16384, 2)
saving arr (28714, 50)
batch selected 85 (16384, 2)
saving arr (28728, 50)
batch selected 86 (16384, 2)
saving arr (28856, 50)
batch selected 87 (16384, 2)
saving arr (28688, 50)
batch selected 88 (16384, 2)
saving arr (28712, 50)
batch selected 89 (16384, 2)
saving arr (28776, 50)
batch selected 90 (16384, 2)
saving arr (28782, 50)
batch selected 91 (16384, 2)
saving arr (28758, 50)
batch selected 92 (16384, 2)
saving arr (28814, 50)
batch selected 93 (16384, 2)
saving arr (28726, 50)
batch select

In [27]:
# import matplotlib.pyplot as plt

# plt.hist(review_sizes.items())

# plt.show()

In [28]:
#review_sizes

In [29]:
_ = np.random.randint(batch.shape[0])
print(batch.iloc[_]["clean_text"])
for w in arr[_,]:
    print(reverse_lookup[w])

this product suck it will look bad in a matter of day with normal usage you will see small scratch which will drive you crazy get the lifetime protector from best buy 25 dollar
this
product
suck
it
will
look
bad
in
matter
day
with
normal
usage
you
will
see
small
scratch
which
will
drive
you
crazy
get
the
lifetime
protector
from
best
buy
dollar
pawn
cobblestone_alleyways
decree_appointing
shrimpers
city_slickers
tentatively_slated
sociétés
revolver_sawed_off
groupage
perkier
3years
banners
toothpaste_toothbrushes
vomit_inducing
toilets_sinks
signaling_pathways
walkers_joggers
hotch_potch
swordfishing


In [30]:
arr.shape

(28560, 50)

In [31]:
sentiment = labels[:,0]
sentiment.shape

(16384,)

In [32]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(arr,  sentiment, test_size=1.0/6, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [28560, 16384]

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_data, train_labels)
preds = lr.predict(test_data)
print("accuracy", np.mean(preds==test_labels))
print("balance", np.mean(test_labels))

In [ ]:
from scipy import stats

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_data, train_labels)
rf_preds = rf.predict(test_data)
print("rf accuracy", np.mean(preds==test_labels))